In [1]:
%matplotlib inline
from pyimagesearch.transform import four_point_transform
from PIL import Image
import pytesseract
import cv2
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import argparse
import imutils
import skimage
from skimage.filters import threshold_local

In [2]:
file_name = 'images/input/pingle.png'
method = ''

In [3]:
def display_image(img):

    dpi = 80


    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)

    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])

    # Hide spines, ticks, etc.
    ax.axis('off')

    # Display the image.
    ax.imshow(img, cmap='gray')

    plt.show()

In [ ]:
def getEdges(img):
    ##(2) convert to hsv-space, then split the channels
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h,s,v = cv2.split(hsv)

    ##(3) threshold the S channel using adaptive method(`THRESH_OTSU`) or fixed thresh
    th, threshed = cv2.threshold(s, 50, 255, cv2.THRESH_BINARY_INV)
    
    
#     # convert the image to grayscale, blur it, and find edges
#     # in the image
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     gray = cv2.GaussianBlur(gray, (5, 5), 0)
#     edged = cv2.Canny(gray, 75, 200)

    # show the original image and the edge detected image
    print("STEP 1: Edge Detection")
    display_image(edged)
    return edged

In [ ]:
def findContours(orig, img, edged):
    # Getting contours  	
    im2, contours, hierarchy = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)	
    # Finding contour of biggest rectangle	# Otherwise return corners of original image	# Don't forget on our 5px border!	height = edges.shape[0]	width = edges.shape[1]	MAX_COUNTOUR_AREA = (width - 10) * (height - 10)	
    # Page fill at least half of image, then saving max area found	
    maxAreaFound = MAX_COUNTOUR_AREA * 0.5
    # Saving page contour	
    pageContour = np.array([[5, 5], [5, height-5], [width-5, heigh-5], [widt-5, 5]])	
    # Go through all contours
    for cnt in contours:
        # Simplify contour
        perimeter = cv2.arcLength(cnt, True)    
        approx = cv2.approxPolyDP(cnt, 0.03 * perimeter, True)
        # Page has 4 corners and it is convex	    # Page area must be bigger than maxAreaFound 	    if (len(approx) == 4 and	            cv2.isContourConvex(approx) and	            maxAreaFound < cv2.contourArea(approx) < MAX_COUNTOUR_AREA):	
        maxAreaFound = cv2.contourArea(approx)	        
        pageContour = approx



    # apply the four point transform to obtain a top-down
    # view of the original image
    warped = four_point_transform(orig, pageContour[:, 0] * ratio)

    # convert the warped image to grayscale, then threshold it
    # to give it that 'black and white' paper effect
    warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
    T = threshold_local(warped, 11, offset = 10, method = "gaussian")
    warped = (warped > T).astype("uint8") * 255

    # show the original and scanned images
    print("STEP 3: Apply perspective transform")
    #display_image(warped)
    
    return warped

In [ ]:
def getString(img_path):
    # Extract the file name without the file extension
    file_name = os.path.basename(img_path).split('.')[0]
    file_name = file_name.split()[0]

    # Create a directory for outputs
    output_path = os.path.join('images/output/', file_name)
    if not os.path.exists(output_path):
        os.makedirs(output_path)

In [ ]:
def rescale(img):
     # Rescale the image, if needed.
    img = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)
    #method += "RS_"
    return img

In [ ]:
def removeNoise(img):
    # Convert to gray
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply dilation and erosion to remove some noise
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)
    img = cv2.erode(img, kernel, iterations=1)
    # Apply blur to smooth out the edges
    img = cv2.GaussianBlur(img, (5, 5), 0)
    #method += "RN_"
    return img

In [ ]:
def binarization(img):
    # Apply threshold to get image with only b&w (binarization)
    img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    #method += "BN_"
    return img

In [ ]:
def saveResult(img):
    # Save the filtered image in the output directory
    save_path = os.path.join('images/output/', file_name + "_filter_.jpg")
    cv2.imwrite(save_path, img)

    # Recognize text with tesseract for python
    result = pytesseract.image_to_string(img, lang="eng")
    return result

In [ ]:
# Read image using opencv
img = cv2.imread(file_name)
height, width, depth = img.shape
display_image(img)

# load the image and compute the ratio of the old height
# to the new height, clone it, and resize it
ratio = img.shape[0] / 500.0
orig = img.copy()
img = imutils.resize(img, height = 800)
    
getString(file_name)

In [ ]:
edged = getEdges(img)

In [ ]:
img = findContours(orig, img, edged)

In [ ]:
display_image(img)

In [ ]:
img = rescale(img)
#img = removeNoise(img)
img = binarization(img)
display_image(img)


In [ ]:
results = saveResult(img)
print(results)